<a href="https://colab.research.google.com/github/abala2017/TitanicMLPredictor/blob/master/Titanic_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.python.data import Dataset
import scipy.optimize as sci

In [0]:
def sigmoid(m):
  z = 1/(1 + np.exp(-m)) 
  return z

In [571]:
test_data =pd.read_csv("https://drive.google.com/uc?export=download&id=1SOvz5VEmzKutGK4MGolDEk15NV2u3Mjm", sep=",")
train_data =pd.read_csv("https://drive.google.com/uc?export=download&id=14L86YTyGwbuJAp10-doHFQ5TNuyIdX2v", sep=",")
train_data["Age"] = train_data["Age"].apply(lambda val: 30.27 if(math.isnan(val)) else val )
test_data["Age"] = test_data["Age"].apply(lambda val: 30.27 if(math.isnan(val)) else val )
train_data["Gender"] = train_data["Sex"].apply(lambda val: 1 if(val == "female") else 0 )
test_data["Gender"] = test_data["Sex"].apply(lambda val: 1 if(val == "female") else 0 )
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.812525,0.523008,0.381594,32.204208,0.352413
std,257.353842,0.486592,0.836071,13.004012,1.102743,0.806057,49.693429,0.477990
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000
50%,446.000000,0.000000,3.000000,30.270000,0.000000,0.000000,14.454200,0.000000
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [572]:
test_data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Gender
count,418.000000,418.000000,418.000000,418.000000,418.000000,417.000000,418.000000
mean,1100.500000,2.265550,30.272057,0.447368,0.392344,35.627188,0.363636
std,120.810458,0.841838,12.634534,0.896760,0.981429,55.907576,0.481622
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,23.000000,0.000000,0.000000,7.895800,0.000000
50%,1100.500000,3.000000,30.270000,0.000000,0.000000,14.454200,0.000000
75%,1204.750000,3.000000,35.750000,1.000000,0.000000,31.500000,1.000000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200,1.000000


In [573]:
t_data = train_data.head(790)
t_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
count,790.000000,790.000000,790.000000,790.000000,790.000000,790.000000,790.000000,790.000000
mean,395.500000,0.387342,2.303797,29.938127,0.513924,0.374684,33.177146,0.353165
std,228.197648,0.487451,0.837257,13.026859,1.035092,0.803268,51.795839,0.478256
min,1.000000,0.000000,1.000000,0.670000,0.000000,0.000000,0.000000,0.000000
25%,198.250000,0.000000,2.000000,22.000000,0.000000,0.000000,7.925000,0.000000
50%,395.500000,0.000000,3.000000,30.270000,0.000000,0.000000,14.500000,0.000000
75%,592.750000,1.000000,3.000000,35.000000,1.000000,0.000000,31.275000,1.000000
max,790.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [574]:
v_data = train_data.tail(100)
validation_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Gender
count,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,100.000000
mean,841.500000,0.360000,2.340000,28.81570,0.600000,0.440000,24.762540,0.350000
std,29.011492,0.482418,0.831452,12.90927,1.543968,0.832666,27.373013,0.479372
min,792.000000,0.000000,1.000000,0.42000,0.000000,0.000000,0.000000,0.000000
25%,816.750000,0.000000,2.000000,21.75000,0.000000,0.000000,7.895800,0.000000
50%,841.500000,0.000000,3.000000,30.27000,0.000000,0.000000,13.000000,0.000000
75%,866.250000,1.000000,3.000000,34.12500,1.000000,1.000000,30.000000,1.000000
max,891.000000,1.000000,3.000000,74.00000,8.000000,5.000000,164.866700,1.000000


In [0]:
def Features(data):
  "Get the data for x ready"
  data_Array = pd.DataFrame()
  data_Array["firstClass"] = data["Pclass"].apply(lambda val: 1 if(val==1) else 0)
  data_Array["secondClass"] = data["Pclass"].apply(lambda val: 1 if(val==2) else 0)
  data_Array["thirdClass"] = data["Pclass"].apply(lambda val: 1 if(val==3) else 0)
  data_Array["Age"] = data["Age"]
  data_Array["SibSp"] = data["SibSp"]
  data_Array["Parch"] = data["Parch"]
  data_Array["Fare"] = data["Fare"]
  data_Array["Male"] = data["Sex"].apply(lambda val: 1 if(val == "male") else 0)
  data_Array["Female"] = data["Sex"].apply(lambda val: 1 if(val == "female") else 0)
  return data_Array

In [0]:
def Labels(data):
  data_Array = pd.DataFrame()
  data_Array["Survived"] = data["Survived"].apply(lambda val: 1 if(val==1) else 0)
  return data_Array

In [0]:
t_features = Features(t_data)
t_labels = Labels(t_data)
v_features = Features(v_data).to_numpy()
v_labels = Labels(v_data).to_numpy()

In [0]:
def CostFunction(theta,X,y):
  m = X.shape[0]  #training examples
  J = 1/m * np.sum(-y * np.matrix.transpose(np.log(sigmoid(X.dot(theta))))- (1-y) * np.matrix.transpose(np.log(1- sigmoid(X.dot(theta)))))
  return J

In [0]:
def Gradient(theta, X, y):
  m = X.shape[0]
  #grad = 1/m * (np.matrix.transpose(((np.matrix.transpose((sigmoid(X * np.matrix.transpose(theta)))))-y)) * X)
  y = y.reshape(m,1)
  grad = 1/m * np.matrix.transpose(sigmoid(X.dot(theta)) - y).dot(X)
  return grad

In [580]:
theta = np.zeros([t_features.shape[1],1])
X = t_features.to_numpy()
Y = t_labels.to_numpy()
optimumVal = sci.minimize(fun = CostFunc1, x0= theta,args=(X,Y), jac = Gradient1)
theta = optimumVal.x

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply


In [0]:
def predict(theta,X,Y):
  predicted = sigmoid(X.dot(theta))
  return predicted

In [0]:
def getAccuracy(theta,X,y):
  m = X.shape[0]
  predicted = predict(theta,X,y)
  is1 = lambda val: 1 if(val>=.5) else 0
  is1Vectorized = np.vectorize(is1)
  predicted = is1Vectorized(predicted)
  y = np.array(y.T)
  isEqual = lambda val1,val2: 1 if(val1==val2) else 0
  isEqualVectorized = np.vectorize(isEqual)
  accuracyVector = isEqualVectorized(predicted,y)
  accuracy = (np.sum(accuracyVector)/m)*100
  return accuracy

In [606]:
# Validation test
getAccuracy(theta,v_features,v_labels)

82.0